In [2]:
# ==============================================================================
# Cell 1: Imports & Environment Setup
# ==============================================================================
import lyricsgenius
import pandas as pd
import os
import time
import json
from dotenv import load_dotenv

# 1. Load Environment Variables
# This looks for a .env file in the root directory
# load_dotenv() returns True if it finds the file
if not load_dotenv(dotenv_path='../.env'):
    print("⚠️ WARNING: .env file not found in root directory.")
else:
    print("✅ Environment variables loaded.")

# 2. Define File Paths
# We use relative paths so this works on any machine
RAW_DATA_PATH = '../data/raw'
PROCESSED_DATA_PATH = '../data/processed'

# 3. Create Directories (Safety Check)
os.makedirs(RAW_DATA_PATH, exist_ok=True)
os.makedirs(PROCESSED_DATA_PATH, exist_ok=True)

print(f"Raw Data Path: {RAW_DATA_PATH}")
print(f"Processed Data Path: {PROCESSED_DATA_PATH}")

✅ Environment variables loaded.
Raw Data Path: ../data/raw
Processed Data Path: ../data/processed


In [3]:
# ==============================================================================
# Cell 2: API Authentication
# ==============================================================================
# Retrieve the token securely from the environment
GENIUS_ACCESS_TOKEN = os.getenv("GENIUS_ACCESS_TOKEN")

if not GENIUS_ACCESS_TOKEN:
    raise ValueError("❌ Error: GENIUS_ACCESS_TOKEN not found in .env file!")

print("Attempting to authenticate with Genius API...")

try:
    # Initialize Genius Object
    # timeout=15: Gives the API 15 seconds before giving up (good for slow connections)
    # sleep_time=0.5: Pauses half a second between calls to be polite
    # retries=3: Tries 3 times if the API fails before throwing an error
    genius = lyricsgenius.Genius(GENIUS_ACCESS_TOKEN, 
                                 timeout=15, 
                                 retries=3, 
                                 sleep_time=0.5,
                                 verbose=True)
    
    # Clean up output: Don't print massive blocks of text headers
    genius.remove_section_headers = True 
    
    print("✅ Genius Client Authenticated Successfully.")
    
except Exception as e:
    print(f"❌ Authentication Failed: {e}")

Attempting to authenticate with Genius API...
✅ Genius Client Authenticated Successfully.


In [4]:
# ==============================================================================
# Cell 3: Configuration (Seed Artists)
# ==============================================================================
# These artists act as the "centers of gravity" for our communities.
# We explicitly choose diverse artists to ensure we find distinct clusters.

SEED_ARTISTS = [
    "Kendrick Lamar",  # West Coast / Lyrical / TDE
    "Drake",           # Global / Pop-Rap / OVO
    "Future",          # Atlanta / Trap / Dungeon Family
    "J. Cole",         # East Coast / Storytelling / Dreamville
    "Kanye West"       # Chicago / Production Hub
]

# Parameters
# Start with 20-30 for testing. Increase to 50 for the full paper.
SONGS_PER_ARTIST = 30 

print(f"Targeting {len(SEED_ARTISTS)} seed artists.")
print(f"Collecting top {SONGS_PER_ARTIST} songs per artist.")

Targeting 5 seed artists.


In [5]:
# ==============================================================================
# Cell 4: The Crawl Function
# ==============================================================================
def crawl_artist_data(artist_name, max_songs):
    """
    Searches for an artist, downloads their top songs/lyrics, 
    and saves the raw object to a JSON file.
    """
    print(f"\n--- Starting crawl for: {artist_name} ---")
    
    # Create a clean filename (e.g., "kendrick_lamar_data.json")
    clean_name = artist_name.replace(" ", "_").lower()
    file_path = os.path.join(RAW_DATA_PATH, f"{clean_name}_data.json")
    
    # CACHING STRATEGY:
    # If we already downloaded this artist, skip them to save time/API calls.
    if os.path.exists(file_path):
        print(f"⚡ Cache hit: Data for '{artist_name}' already exists.")
        print(f"   Loading from {file_path}...")
        with open(file_path, 'r') as f:
            # We load it just to return the object, but we don't re-download
            return json.load(f)
    
    # If not in cache, download from Genius
    try:
        # 'sort="popularity"' ensures we get the most important songs first
        artist = genius.search_artist(artist_name, max_songs=max_songs, sort="popularity")
        
        if artist:
            # Save the raw data immediately
            # This saves EVERYTHING (lyrics, metadata, features) in one file
            artist.save_lyrics(filename=file_path, overwrite=True, verbose=True)
            print(f"✅ Download complete. Saved to {file_path}")
            return artist
        else:
            print(f"⚠️ Artist '{artist_name}' not found.")
            return None
        
    except Exception as e:
        print(f"❌ Error crawling {artist_name}: {e}")
        return None

In [6]:
# ==============================================================================
# Cell 5: Execution Loop
# ==============================================================================
# This loop runs the crawl function for every seed artist.

print("🚀 Starting Data Collection Phase...")

for artist_name in SEED_ARTISTS:
    crawl_artist_data(artist_name, SONGS_PER_ARTIST)
    
    # Safety pause to prevent rate-limiting
    print("Sleeping for 2 seconds...")
    time.sleep(2)

print("\n✅ Phase 1 Crawl Complete.")
print(f"Check the '{RAW_DATA_PATH}' folder to see your JSON files.")

🚀 Starting Data Collection Phase...

--- Starting crawl for: Kendrick Lamar ---
Searching for songs by Kendrick Lamar...

Song 1: "Not Like Us"
Song 2: "HUMBLE."
Song 3: "euphoria"
Song 4: "m.A.A.d city"
Song 5: "Swimming Pools (Drank)"
Song 6: "Money Trees"
Song 7: "DNA."
Song 8: "XXX."
Song 9: "meet the grahams"
Song 10: "Bitch, Don’t Kill My Vibe"
Song 11: "King Kunta"
Song 12: "Poetic Justice"
Song 13: "Alright"
Song 14: "LOVE."
Song 15: "Backseat Freestyle"
Song 16: "Sing About Me, I’m Dying of Thirst"
Song 17: "The Blacker the Berry"
Song 18: "ELEMENT."
Song 19: "LOYALTY."
Song 20: "i"
Song 21: "Wesley’s Theory"
Song 22: "All The Stars"
Song 23: "PRIDE."
Song 24: "FEAR."
Song 25: "6:16 in LA"
Song 26: "A.D.H.D"
Song 27: "Mortal Man"
Song 28: "u"
Song 29: "tv off"
Song 30: "DUCKWORTH."

Reached user-specified song limit (30).
Done. Found 30 songs.
Wrote ../data/raw/kendrick_lamar_data.json.
✅ Download complete. Saved to ../data/raw/kendrick_lamar_data.json
Sleeping for 2 seconds..

In [7]:
# ==============================================================================
# Cell 6: Data Processing (JSON to CSV)
# ==============================================================================
# Now we turn the raw JSON files into clean tables (CSVs) 
# for Phase 2 (Network) and Phase 3 (NLP).

network_edges = []
song_lyrics = []

print("🔄 Processing raw JSON files into structured CSVs...")

# Iterate through every file in the raw data folder
for filename in os.listdir(RAW_DATA_PATH):
    if filename.endswith(".json"):
        filepath = os.path.join(RAW_DATA_PATH, filename)
        
        # Load the JSON file
        with open(filepath, 'r') as f:
            data = json.load(f)
            
        # The JSON structure from lyricsgenius is: {'name': 'Artist', 'songs': [...]}
        main_artist = data['name']
        songs = data['songs']
        
        print(f"   Processing {len(songs)} songs for {main_artist}...")
        
        for song in songs:
            title = song['title']
            lyrics = song['lyrics']
            release_date = song.get('release_date_for_display', 'N/A')
            
            # --- DATASET 1: LYRICS (For Assignment 3) ---
            # We only keep songs that actually have lyrics
            if lyrics and len(lyrics) > 50:
                song_lyrics.append({
                    'artist': main_artist,
                    'title': title,
                    'year': release_date,
                    'lyrics': lyrics
                })
            
            # --- DATASET 2: NETWORK EDGES (For Assignment 2) ---
            # We look for featured artists to create connections
            if song['featured_artists']:
                for feature in song['featured_artists']:
                    feature_name = feature['name']
                    
                    # Add the edge: Source -> Target
                    network_edges.append({
                        'source': main_artist,
                        'target': feature_name,
                        'song': title
                    })

# Convert lists to Pandas DataFrames
df_edges = pd.DataFrame(network_edges)
df_lyrics = pd.DataFrame(song_lyrics)

# Save to CSV
edges_path = os.path.join(PROCESSED_DATA_PATH, 'network_edges.csv')
lyrics_path = os.path.join(PROCESSED_DATA_PATH, 'song_lyrics.csv')

df_edges.to_csv(edges_path, index=False)
df_lyrics.to_csv(lyrics_path, index=False)

print("\n✅ PROCESSING COMPLETE")
print(f"   Network Edges saved: {len(df_edges)} rows -> {edges_path}")
print(f"   Song Lyrics saved:   {len(df_lyrics)} rows -> {lyrics_path}")

# Show a preview to confirm
print("\n--- Preview: Network Edges ---")
print(df_edges.head())

🔄 Processing raw JSON files into structured CSVs...
   Processing 30 songs for J. Cole...
   Processing 30 songs for Drake...
   Processing 30 songs for Kendrick Lamar...
   Processing 30 songs for Kanye West...
   Processing 30 songs for Future...

✅ PROCESSING COMPLETE
   Network Edges saved: 69 rows -> ../data/processed/network_edges.csv
   Song Lyrics saved:   150 rows -> ../data/processed/song_lyrics.csv

--- Preview: Network Edges ---
    source          target              song
0  J. Cole   Amber Coffman         She Knows
1  J. Cole          Miguel        Power Trip
2  J. Cole             TLC     Crooked Smile
3  J. Cole  Kendrick Lamar   Forbidden Fruit
4  J. Cole   Missy Elliott  Nobody’s Perfect
